In [2]:
# randomly sample 500 movies from kaggle dataset of all movies
import pandas as pd

df = pd.read_csv('top_rated_movies.csv')

print(df.columns)

df_sampled = df.sample(n=500, random_state=1)  # n = 500 movies

df_sampled['overview'] = df_sampled['overview'].fillna('').str.lower()

df_sampled.to_csv('sampled_movies.csv', index=False)

ModuleNotFoundError: No module named 'pandas'

In [21]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def load_data(filepath):
    """
    Load dataset containing item descriptions.
    """
    return pd.read_csv(filepath)

def preprocess_data(df, text_column):
    """
    Preprocess the text data by filling nulls and converting to lowercase.
    """
    df[text_column] = df[text_column].fillna('').str.lower()
    return df

def build_tfidf_matrix(df, text_column):
    """
    Convert text data into TF-IDF vectors.
    """
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(df[text_column])
    return tfidf_matrix, vectorizer

def get_recommendations(user_input, tfidf_matrix, vectorizer, df, top_n=5):
    """
    Recommend top N items based on cosine similarity.
    """
    user_tfidf = vectorizer.transform([user_input.lower()])
    cosine_sim = cosine_similarity(user_tfidf, tfidf_matrix).flatten()
    top_indices = cosine_sim.argsort()[-top_n:][::-1]
    return df.iloc[top_indices][['title', 'overview']], cosine_sim[top_indices]



In [22]:
if __name__ == "__main__":
    # Load and preprocess data
    filepath = 'sampled_movies.csv'  # Update with correct path
    df = load_data(filepath)

    df = preprocess_data(df, 'overview')
    
    # Build TF-IDF matrix
    tfidf_matrix, vectorizer = build_tfidf_matrix(df, 'overview')
    
    # Get user input
    user_input = input("Enter a short text description of your preferences: ")
    
    # Get recommendations
    recommendations, scores = get_recommendations(user_input, tfidf_matrix, vectorizer, df)
    
    # Display results
    print("Top recommendations:")
    for i, (index, row) in enumerate(recommendations.iterrows()):
        print(f"{i+1}. {row['title']} (Score: {scores[i]:.4f})")


Top recommendations:
1. Space Chimps (Score: 0.2222)
2. High Life (Score: 0.1794)
3. Rifkin’s Festival (Score: 0.1644)
4. The Last Temptation of Christ (Score: 0.1215)
5. Dragon: The Bruce Lee Story (Score: 0.1106)
